# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [1]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data

training_file = 'train.p'
testing_file = 'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']
c_train, c_test = train['coords'], test['coords']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 2D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below.

In [2]:
import csv

sign_names = []
with open('signnames.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        sign_names.append(row[1])
    sign_names.reverse()
    sign_names.pop()
    sign_names.reverse()

In [3]:
### Replace each question mark with the appropriate value.

# TODO: Number of training examples
assert(len(X_train) == len(y_train))
n_train = len(X_train)

# TODO: Number of testing examples.
assert(len(X_test)==len(y_test))
n_test = len(X_test)

# TODO: What's the shape of an traffic sign image?
image_shape = X_train[0].shape

# TODO: How many unique classes/labels there are in the dataset.
n_classes = 43

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Number of training examples = 39209
Number of testing examples = 12630
Image data shape = (32, 32, 3)
Number of classes = 43


Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

In [ ]:
### Data exploration visualization goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import random
# Visualizations will be shown in the notebook.
%matplotlib inline

num_x = 5
num_y = 5
fig, ax = plt.subplots(num_y,num_x)
fig.set_size_inches(20,20)
for i in range(num_x*num_y):
    n = random.random()*n_train
    ax[int(i/num_x)][i%num_x].imshow(X_train[n])
    coords = c_train[n]
    rect = patches.Rectangle((coords[0],coords[1]),coords[2]-coords[0],coords[3]-coords[1],edgecolor='#00ff00',facecolor='none')
    ax[int(i/num_x)][i%num_x].add_patch(rect)
    ax[int(i/num_x)][i%num_x].set_title(sign_names[y_train[n]])

plt.axis('off')
plt.show()

In [ ]:
import numpy as np
hist_data = np.histogram(y_train,bins=range(44))
hist_map = {}
for occr,index in zip(hist_data[0],hist_data[1]):
    hist_map[occr]=index

plt.hist(y_train,bins=range(44),color='#00ffCC')
plt.axis([0,43,0,2500])
plt.show()

print("Maximum Occuring Traffic Sign in the data: "+sign_names[hist_map[np.amax(hist_data[0])]])
print("Minimum Occuring Traffic Sign in the data: "+sign_names[hist_map[np.amin(hist_data[0])]])

In [4]:
import numpy as np

X_train = np.array(X_train)
X_train = X_train/255

In [5]:
chann_index_swap = np.swapaxes(X_train,0,3)
mean_image = [[[sum(pixel)/len(pixel) for pixel in col] for col in row] for row in chann_index_swap]
mean_image = np.swapaxes(mean_image,0,2)
mean_image = np.swapaxes(mean_image,0,1)

X_train = X_train - mean_image

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(mean_image)
plt.show()

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Neural network architecture
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

**NOTE:** The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play!

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [6]:
### Preprocess the data here.
### Feel free to use as many code cells as needed.
import tensorflow as tf

EPOCHS = 10
BATCH_SIZE = 128

In [7]:
### Preprocess the data here.
### Feel free to use as many code cells as need
def conv2d(x,W,b,output_depth,train=True,strides=1):
    x = tf.nn.conv2d(x,W,strides=[1,strides,strides,1],padding='VALID')
    x = batch_norm(x,output_depth,train)
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)   

In [8]:
def max_pool(x,k=2):
    return tf.nn.max_pool(x,ksize=[1,k,k,1],strides=[1,k,k,1],padding='VALID')

In [9]:
depth = {
    '1_c': 30,
    '2_c': 200,
    '3_f': 2200,
    '4_f': 1000,
    '5_f': 500,
    '6_f': 120,
    '7_o': 43
}

In [10]:
mu  = 0
SD = 0.1
weights = {
    #'1_conv': tf.Variable(tf.truncated_normal(shape=(5,5,3,depth['1_c']), mean = mu, stddev = SD)),
    '1_conv': tf.get_variable("W_1_conv", shape=[5, 5, 3, depth['1_c']],initializer=tf.contrib.layers.xavier_initializer()),
    
    #'2_conv': tf.Variable(tf.truncated_normal(shape=(5,5,depth['1_c'],depth['2_c']), mean = mu, stddev = SD)),
    '2_conv': tf.get_variable("W_2_conv", shape=[5, 5, depth['1_c'], depth['2_c']],initializer=tf.contrib.layers.xavier_initializer()),
    
    #'3_fullc': tf.Variable(tf.truncated_normal(shape=(5000,depth['3_f']),mean = mu, stddev = SD)),
    '3_fullc': tf.get_variable("W_3_fullc", shape=[5000, depth['3_f']],initializer=tf.contrib.layers.xavier_initializer()),
    
    #'4_fullc': tf.Variable(tf.truncated_normal(shape=(depth['3_f'],depth['4_f']),mean = mu, stddev = SD)),
    '4_fullc': tf.get_variable("W_4_fullc", shape=[depth['3_f'], depth['4_f']],initializer=tf.contrib.layers.xavier_initializer()),
    
    #'5_fullc': tf.Variable(tf.truncated_normal(shape=(depth['4_f'],depth['5_f']),mean = mu, stddev = SD)),
    '5_fullc': tf.get_variable("W_5_fullc", shape=[depth['4_f'], depth['5_f']],initializer=tf.contrib.layers.xavier_initializer()),
    
    #'6_fullc': tf.Variable(tf.truncated_normal(shape=(depth['5_f'],depth['6_f']),mean = mu, stddev = SD)),
    '6_fullc': tf.get_variable("W_6_fullc", shape=[depth['5_f'], depth['6_f']],initializer=tf.contrib.layers.xavier_initializer()),
    
    #'7_out': tf.Variable(tf.truncated_normal(shape=(depth['6_f'],depth['7_o']), mean = mu, stddev = SD))
    '7_out': tf.get_variable("W_7_out", shape=[depth['6_f'], depth['7_o']],initializer=tf.contrib.layers.xavier_initializer())
}

In [11]:
biases = {
    '1_conv': tf.Variable(tf.zeros(depth['1_c'])),
    '2_conv': tf.Variable(tf.zeros(depth['2_c'])),
    '3_fullc': tf.Variable(tf.zeros(depth['3_f'])),
    '4_fullc': tf.Variable(tf.zeros(depth['4_f'])),
    '5_fullc': tf.Variable(tf.zeros(depth['5_f'])),
    '6_fullc': tf.Variable(tf.zeros(depth['6_f'])),
    '7_out': tf.Variable(tf.zeros(depth['7_o']))
}

In [12]:
def batch_norm(x, n_out, train):
    """
    Batch normalization on convolutional maps.
    Ref.: http://stackoverflow.com/questions/33949786/how-could-i-use-batch-normalization-in-tensorflow
    Args:
        x:           Tensor, 4D BHWD input maps
        n_out:       integer, depth of input maps
        phase_train: boolean tf.Varialbe, true indicates training phase
        scope:       string, variable scope
    Return:
        normed:      batch-normalized maps
    """
    with tf.variable_scope('bn'):
        beta = tf.Variable(tf.constant(0.0, shape=[n_out]),
                                     name='beta', trainable=True)
        gamma = tf.Variable(tf.constant(1.0, shape=[n_out]),
                                      name='gamma', trainable=True)
        batch_mean, batch_var = tf.nn.moments(x, [0,1,2], name='moments')
        ema = tf.train.ExponentialMovingAverage(decay=0.5)

        def mean_var_with_update():
            ema_apply_op = ema.apply([batch_mean, batch_var])
            with tf.control_dependencies([ema_apply_op]):
                return tf.identity(batch_mean), tf.identity(batch_var)

        mean, var = tf.cond(train,
                            mean_var_with_update,
                            lambda: (ema.average(batch_mean), ema.average(batch_var)))
        normed = tf.nn.batch_normalization(x, mean, var, beta, gamma, 1e-3)
    return normed

In [13]:
def batch_norm_fullc(x, n_out, train):
    
    with tf.variable_scope('bn'):
        beta = tf.Variable(tf.constant(0.0, shape=[n_out]),
                                     name='beta', trainable=True)
        gamma = tf.Variable(tf.constant(1.0, shape=[n_out]),
                                      name='gamma', trainable=True)
        batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
        ema = tf.train.ExponentialMovingAverage(decay=0.5)

        def mean_var_with_update():
            ema_apply_op = ema.apply([batch_mean, batch_var])
            with tf.control_dependencies([ema_apply_op]):
                return tf.identity(batch_mean), tf.identity(batch_var)

        mean, var = tf.cond(train,
                            mean_var_with_update,
                            lambda: (ema.average(batch_mean), ema.average(batch_var)))
        normed = tf.nn.batch_normalization(x, mean, var, beta, gamma, 1e-3)
    return normed

In [14]:
from tensorflow.contrib.layers import flatten

def LeNet(x,train=True):    
    # Hyperparameters
    mu = 0
    sigma = 0.1
    
    # TODO: Layer 1: Convolutional. Input = 32x32x3. Output = 30x30x12.

    # TODO: Activation.
    conv1 = conv2d(x,weights['1_conv'],biases['1_conv'],depth['1_c'],train)

    # TODO: Pooling. Input = 30x30x12. Output = 15x15x12.
    conv1 = max_pool(conv1)
    #conv1 = batch_norm_layer(conv1,train_phase,scope_bn='bn')
    
    #conv1 = tf.cond(dropout > 0, lambda: tf.nn.dropout(conv1,0.98), lambda: conv1)
    # TODO: Layer 2: Convolutional. Output = 13x13x48.
    
    # TODO: Activation.
    conv2 = conv2d(conv1,weights['2_conv'],biases['2_conv'],depth['2_c'],train)

    # TODO: Pooling. Input = 13x13x48. Output = 7x7x48.
    conv2 = max_pool(conv2)
    
    #conv2 = batch_norm_layer(conv2,train_phase,scope_bn='bn')
    
    #conv2 = tf.cond(dropout > 0, lambda: tf.nn.dropout(conv2,0.92), lambda: conv2)
    
    # Flatten: Input = 2x2x192 Output = 768
    fc0 = flatten(conv2)
    
    # TODO: Layer 4: Fully Connected. Input = 768. Output = 84.
    fullc1 = tf.add(tf.matmul(fc0,weights['3_fullc']),biases['3_fullc'])
    fullc1 = batch_norm_fullc(fullc1,depth['3_f'],train)
    # TODO: Activation.
    fullc1 = tf.nn.relu(fullc1)

    fullc1 = tf.cond(train, lambda: tf.nn.dropout(fullc1,0.50), lambda: fullc1)
    
    fullc2 = tf.add(tf.matmul(fullc1,weights['4_fullc']),biases['4_fullc'])
    fullc2 = batch_norm_fullc(fullc2,depth['4_f'],train)
    fullc2 = tf.nn.relu(fullc2)
    
    fullc2 = tf.cond(train, lambda: tf.nn.dropout(fullc2,0.50), lambda: fullc2)
    
    fullc3 = tf.add(tf.matmul(fullc2,weights['5_fullc']),biases['5_fullc'])
    fullc3 = batch_norm_fullc(fullc3,depth['5_f'],train)
    fullc3 = tf.nn.relu(fullc3)
    
    fullc3 = tf.cond(train, lambda: tf.nn.dropout(fullc3,0.50), lambda: fullc3)
    
    fullc4 = tf.add(tf.matmul(fullc3,weights['6_fullc']),biases['6_fullc'])
    fullc4 = batch_norm_fullc(fullc4,depth['6_f'],train)
    fullc4 = tf.nn.relu(fullc4)
    
    fullc4 = tf.cond(train, lambda: tf.nn.dropout(fullc4,0.50), lambda: fullc4)
    
    # TODO: Layer 6: Fully Connected. Input = 84. Output = 10.
    logits = tf.add(tf.matmul(fullc4,weights['7_out']),biases['7_out'])
    
    return logits

### Question 1 

_Describe how you preprocessed the data. Why did you choose that technique?_

**Answer:**

In [15]:
### Generate data additional data (OPTIONAL!)
### and split the data into training/validation/testing sets here.
### Feel free to use as many code cells as needed.

### Question 2

_Describe how you set up the training, validation and testing data for your model. **Optional**: If you generated additional data, how did you generate the data? Why did you generate the data? What are the differences in the new dataset (with generated data) from the original dataset?_

**Answer:**

In [16]:
### Define your architecture here.
### Feel free to use as many code cells as needed.
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 43)

In [17]:
rate = tf.placeholder(tf.float32)

train = tf.placeholder(tf.bool)
lr = 0.0001
b = 1e-6
k = 1e-7

logits = LeNet(x,train)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, one_hot_y)
loss = tf.reduce_mean(cross_entropy)

beta = tf.placeholder(tf.float32)

loss = tf.reduce_mean(loss+beta*tf.nn.l2_loss(weights['1_conv']))
loss = tf.reduce_mean(loss+beta*tf.nn.l2_loss(weights['2_conv']))
loss = tf.reduce_mean(loss+beta*tf.nn.l2_loss(weights['3_fullc']))
loss = tf.reduce_mean(loss+beta*tf.nn.l2_loss(weights['4_fullc']))
loss = tf.reduce_mean(loss+beta*tf.nn.l2_loss(weights['5_fullc']))
loss = tf.reduce_mean(loss+beta*tf.nn.l2_loss(weights['6_fullc']))
loss = tf.reduce_mean(loss+beta*tf.nn.l2_loss(weights['7_out']))
loss = tf.reduce_mean(loss+beta*tf.nn.l2_loss(biases['1_conv']))
loss = tf.reduce_mean(loss+beta*tf.nn.l2_loss(biases['2_conv']))
loss = tf.reduce_mean(loss+beta*tf.nn.l2_loss(biases['3_fullc']))
loss = tf.reduce_mean(loss+beta*tf.nn.l2_loss(biases['4_fullc']))
loss = tf.reduce_mean(loss+beta*tf.nn.l2_loss(biases['5_fullc']))
loss = tf.reduce_mean(loss+beta*tf.nn.l2_loss(biases['6_fullc']))
loss_operation = tf.reduce_mean(loss+beta*tf.nn.l2_loss(biases['7_out']))

optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [18]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y,train:False})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [19]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X_train,y_train,test_size=0.2,random_state=0)
X_train, y_train = shuffle(X_train, y_train)

In [ ]:
tf.reset_default_graph()

In [ ]:
import math

small_sample_x = X_train[0:20]
small_sample_y = y_train[0:20]

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    num_examples = len(small_sample_x)
    
    batch_size = 10
    print("Can we overfit?.......")
    print()
    
    print("Decay: "+str(k))
    sess.run(tf.initialize_all_variables())
    for i in range(100):
        lr = float(lr*np.exp(-k*i))
        #small_sample_x, small_sample_y = shuffle(small_sample_x, small_sample_y)
        for offset in range(0,num_examples,batch_size):
            end = offset + batch_size
            batch_x, batch_y = small_sample_x[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x:small_sample_x, y:small_sample_y, train:False, rate:lr, beta:b})
        c = sess.run(loss_operation, feed_dict={x: small_sample_x, y: small_sample_y,train:False,beta:b})
        print("Epoch: "+str(i)+" loss: "+str(c)+" learn rate:"+str(lr)+" beta: "+str(b)+" decay: "+str(k))

In [21]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y,train:True, rate:lr, beta:b})
            
        validation_accuracy = evaluate(X_validation, y_validation)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, 'lenet')
    print("Model saved")

Training...

EPOCH 1 ...
Validation Accuracy = 0.499

EPOCH 2 ...
Validation Accuracy = 0.668

EPOCH 3 ...
Validation Accuracy = 0.777

EPOCH 4 ...
Validation Accuracy = 0.807

EPOCH 5 ...
Validation Accuracy = 0.893

EPOCH 6 ...
Validation Accuracy = 0.889

EPOCH 7 ...
Validation Accuracy = 0.934

EPOCH 8 ...
Validation Accuracy = 0.960

EPOCH 9 ...
Validation Accuracy = 0.964

EPOCH 10 ...
Validation Accuracy = 0.985

Model saved


### Question 3

_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)  For reference on how to build a deep neural network using TensorFlow, see [Deep Neural Network in TensorFlow
](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/b516a270-8600-4f93-a0a3-20dfeabe5da6/concepts/83a3a2a2-a9bd-4b7b-95b0-eb924ab14432) from the classroom._


**Answer:**

In [ ]:
### Train your model here.
### Feel free to use as many code cells as needed.

### Question 4

_How did you train your model? (Type of optimizer, batch size, epochs, hyperparameters, etc.)_


**Answer:**

### Question 5


_What approach did you take in coming up with a solution to this problem? It may have been a process of trial and error, in which case, outline the steps you took to get to the final solution and why you chose those steps. Perhaps your solution involved an already well known implementation or architecture. In this case, discuss why you think this is suitable for the current problem._

**Answer:**

---

## Step 3: Test a Model on New Images

Take several pictures of traffic signs that you find on the web or around you (at least five), and run them through your classifier on your computer to produce example results. The classifier might not recognize some local signs but it could prove interesting nonetheless.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Question 6

_Choose five candidate images of traffic signs and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult? It could be helpful to plot the images in the notebook._



**Answer:**

In [ ]:
### Run the predictions here.
### Feel free to use as many code cells as needed.

### Question 7

_Is your model able to perform equally well on captured pictures when compared to testing on the dataset? The simplest way to do this check the accuracy of the predictions. For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate._

_**NOTE:** You could check the accuracy manually by using `signnames.csv` (same directory). This file has a mapping from the class id (0-42) to the corresponding sign name. So, you could take the class id the model outputs, lookup the name in `signnames.csv` and see if it matches the sign from the image._


**Answer:**

In [ ]:
### Visualize the softmax probabilities here.
### Feel free to use as many code cells as needed.

### Question 8

*Use the model's softmax probabilities to visualize the **certainty** of its predictions, [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. Which predictions is the model certain of? Uncertain? If the model was incorrect in its initial prediction, does the correct prediction appear in the top k? (k should be 5 at most)*

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.